# Notebook 2: Going Deeper

**TurboPanda** has a lot of interesting functionality to do with manipulating columns and groups of columns that `pandas` so often lacks.

In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.insert(0,"../")
# our main import
import turbopanda as turb

## Reading in our dataset

In [2]:
g = turb.read("../../Research Data/datasets/aviner_translation.csv", name="Translation")
g.drop("Majority")
g

MetaPanda(Translation(n=5216, p=15, mem=0.626MB), mode='instant')

## Accessing with `head`

Normally copying over features from `pandas.DataFrame` was a forbidden fruit, but we decided in this case that `head` so useful that we would break tradition:

In [3]:
g.head(3)

colnames,Protein_IDs,Protein_names,Gene_names,Intensity_G1_1,Intensity_G1_2,Intensity_G1_3,Intensity_G2M_1,Intensity_G2M_2,Intensity_G2M_3,Intensity_MG1_1,Intensity_MG1_2,Intensity_MG1_3,Intensity_S_1,Intensity_S_2,Intensity_S_3
counter,,,,,,,,,,,,,,,
0,Q96IC2;Q96IC2-2;H3BM72;H3BV93;H3BSC5,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662,21.05682
1,H0YGH4;P01023;H0YGH6;F8W7L3,Alpha-2-macroglobulin,A2M,22.62015,22.26825,23.11786,24.94606,24.21645,25.26399,23.56139,23.46051,22.21951,22.87688,23.35703,23.25724
2,A8K2U0;F5H2W3;H0YGG5;F5H2Z2;F5GXP1,Alpha-2-macroglobulin-like protein 1,A2ML1,NaN,NaN,NaN,NaN,NaN,25.11629,NaN,NaN,NaN,NaN,NaN,NaN


## Generic `apply` to the underlying DataFrame

For the thousands of instances where we simply want to apply a `pandas.DataFrame.*` function to the underlying dataset, but retain the consistency between `df_` and `meta_` attributes in particular, our solution is to provide an `apply` function which is performed on the entire
dataset.

This is similar to the `transform` function we have, however `apply` only looks for functions within `pandas.DataFrame.*` API, and it does not allow for pre-subset selection (using the `selector` parameter) beforehand.

For instance, let's say we want to fill all the `NaN` values in dataset with a value. We could do this easily with `pandas.DataFrame.fillna` but to use `MetaPanda` we'd have to call the `transform` function with a custom lambda etc.

In [4]:
g.apply("fillna", 0)

MetaPanda(Translation(n=5216, p=15, mem=0.627MB), mode='instant')

It's that easy.

In [5]:
g.df_.head(3)

colnames,Protein_IDs,Protein_names,Gene_names,Intensity_G1_1,Intensity_G1_2,Intensity_G1_3,Intensity_G2M_1,Intensity_G2M_2,Intensity_G2M_3,Intensity_MG1_1,Intensity_MG1_2,Intensity_MG1_3,Intensity_S_1,Intensity_S_2,Intensity_S_3
counter,,,,,,,,,,,,,,,
0,Q96IC2;Q96IC2-2;H3BM72;H3BV93;H3BSC5,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662,21.05682
1,H0YGH4;P01023;H0YGH6;F8W7L3,Alpha-2-macroglobulin,A2M,22.62015,22.26825,23.11786,24.94606,24.21645,25.26399,23.56139,23.46051,22.21951,22.87688,23.35703,23.25724
2,A8K2U0;F5H2W3;H0YGG5;F5H2Z2;F5GXP1,Alpha-2-macroglobulin-like protein 1,A2ML1,0.00000,0.00000,0.00000,0.00000,0.00000,25.11629,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


## String manipulation of ID columns

Often datasets can have numerical/quantitative data coupled with IDs that help to identify rows based on your problem domain. In this case, we have some identifiers from **Uniprot**, one of the main databases managing protein sequences.

There is a slight problem though, each of these expression values has a *stack of protein IDs* associated with it. So we're going to have to untangle this somehow by **expanding** the ID columns to something we can then perform **set theory** operations on.

`MetaPanda` comes with an `expand` function which does exactly this:

In [6]:
g.expand("Protein_IDs", sep=";")

MetaPanda(Translation(n=26318, p=15, mem=3.370MB), mode='instant')

We can see in this instance that $n$ has increased significantly, as has the memory usage.

In [7]:
g.df_.head()

colnames,Protein_IDs,Protein_names,Gene_names,Intensity_G1_1,Intensity_G1_2,Intensity_G1_3,Intensity_G2M_1,Intensity_G2M_2,Intensity_G2M_3,Intensity_MG1_1,Intensity_MG1_2,Intensity_MG1_3,Intensity_S_1,Intensity_S_2,Intensity_S_3
counter,,,,,,,,,,,,,,,
0,Q96IC2,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662,21.05682
0,Q96IC2-2,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662,21.05682
0,H3BM72,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662,21.05682
0,H3BV93,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662,21.05682
0,H3BSC5,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662,21.05682


## Applying a transformation to eliminate duplicates

Here we see that there are labels that are *sort of* duplicated in the sense that they are appended with `-[0-9]`. We will split these off and keep the left-hand part, then apply a function which drops duplicates.

In [8]:
g.transform(lambda x: x.str.split("-",expand=True)[0], "^Protein_ID")

MetaPanda(Translation(n=26318, p=15, mem=3.370MB), mode='instant')

Now use `pandas.DataFrame.drop_duplicates`, with the subset just on Protein IDs

In [9]:
g.apply("drop_duplicates", subset=["Protein_IDs"])

MetaPanda(Translation(n=21050, p=15, mem=2.695MB), mode='instant')

In [10]:
g.df_.head(3)

colnames,Protein_IDs,Protein_names,Gene_names,Intensity_G1_1,Intensity_G1_2,Intensity_G1_3,Intensity_G2M_1,Intensity_G2M_2,Intensity_G2M_3,Intensity_MG1_1,Intensity_MG1_2,Intensity_MG1_3,Intensity_S_1,Intensity_S_2,Intensity_S_3
counter,,,,,,,,,,,,,,,
0,Q96IC2,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662,21.05682
0,H3BM72,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662,21.05682
0,H3BV93,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662,21.05682


## Using `mode=delay`: Delaying actions to make a task graph

You may have noticed that the representation of the object shows `mode='instant'`.

Using the current approach, all of our operations apply instantly. This can be nice but if you make a mistake, it can be difficult to backtrack.

Fortunately, `MetaPanda` has a `mode` parameter during initialization and `mode_` attribute that can be set at any point. When set to `delay`, many functions that apply changes immediately to the data are instead *cached* in a `pipe_` attribute.

This attribute is then emptied when a call to `compute()` is made, and all of the operations within the `pipe_` are executed with their parameters, in order.

In [11]:
g.mode_ = "delay"

In [12]:
g

MetaPanda(Translation(n=21050, p=15, mem=2.695MB), mode='delay')

### Creating a series of operations to perform...

In [13]:
g.drop("Protein_name")
g.apply("groupby", ["counter","Gene_names"])
g.apply("mean")
g.rename([("Intensity","trans")])

We can see that no changes have been made to the dataset.

In [14]:
g

MetaPanda(Translation(n=21050, p=15, mem=2.695MB), mode='delay')

But if we check the `pipe_` attribute, all of the operations have been saved:

In [15]:
g.pipe_

[('drop', ('Protein_name',), {}),
 ('apply', ('groupby', ['counter', 'Gene_names']), {}),
 ('apply', ('mean',), {}),
 ('rename', ([('Intensity', 'trans')],), {})]

### Now a call to `compute()`:

Note that `compute()` will empty the `pipe_` attribute to prevent repeat calls.

Special provisions allow `compute()` to store a `DataFrameGroupBy` object in between the call to `groupby()` and `mean`, this would normally not be possible in `instant` mode.

In [16]:
g.compute()

MetaPanda(Translation(n=5190, p=12, mem=0.602MB), mode='delay')

In [17]:
g.df_.head()

,colnames,trans_G1_1,trans_G1_2,trans_G1_3,trans_G2M_1,trans_G2M_2,trans_G2M_3,trans_MG1_1,trans_MG1_2,trans_MG1_3,trans_S_1,trans_S_2,trans_S_3
counter,Gene_names,,,,,,,,,,,,
0,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662,21.05682
1,A2M,22.62015,22.26825,23.11786,24.94606,24.21645,25.26399,23.56139,23.46051,22.21951,22.87688,23.35703,23.25724
2,A2ML1,0.00000,0.00000,0.00000,0.00000,0.00000,25.11629,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,AAAS,25.48382,24.42746,25.22645,24.44556,23.93706,25.30966,25.61462,25.45923,24.48253,24.31645,23.92143,25.21838
4,AACS,24.18177,24.51533,24.32766,24.15993,24.05001,24.95797,24.11656,24.22523,23.96446,23.89440,23.78107,24.32629


In [18]:
g.pipe_

[]

### Functions that are affected by `delay` (accessed in `turb.metapanda.__delay_functions__`) include:

- `add_prefix`
- `add_suffix`
- `apply`
- `drop`
- `expand`
- `melt`
- `meta_map`
- `multi_transform`
- `rename`
- `shrink`
- `sort_columns`
- `split_categories`
- `transform`